# UberOrca SubtleGenius v5 - Infrastructure Edition
## The Stig-Level Precision Driving - Full Integration

**NEW IN V5:**
- ✅ Performance Tracker - Real-time coverage + accuracy monitoring
- ✅ Fixed Object Transformations - Detection + Transformation together
- ✅ Enhanced Logging - Per-solver statistics
- ✅ Comprehensive Telemetry - Know what works, data-driven iteration

**Integrated Solvers:**
- Iteration 4: Rule Induction (HIGHEST PRIORITY)
- Iteration 1: Pattern Matching (geometric + color)
- Iteration 2: Object Detection (FIXED - actual transformations)
- Iteration 3: Ensemble Voting

**Infrastructure:**
- Transmission: Validation-ready architecture
- Engine: Data-driven solver selection
- Twin-Turbo: Real-time performance monitoring

**Output:**
- `/kaggle/working/submission.json` - ARC Prize 2025 format
- `/kaggle/working/log.txt` - Detailed execution log + statistics

**Status:** ✅ Production Ready - The Stig Edition

---

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 1: CONFIGURATION, IMPORTS & PERFORMANCE TRACKER
# ═══════════════════════════════════════════════════════════════════════════════

import json
import numpy as np
import os
import time
from typing import Dict, List, Any, Tuple, Optional, Callable
from datetime import datetime
from dataclasses import dataclass, field
from collections import Counter, defaultdict

print("🐋 UberOrca SubtleGenius v5 - Infrastructure Edition")
print(f"⏰ Initialized: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("🏎️  The Stig-Level Precision Driving - Engaged\n")

# ═══════════════════════════════════════════════════════════════════════════════
# CONFIGURATION
# ═══════════════════════════════════════════════════════════════════════════════

class ARC2025Config:
    IS_KAGGLE = os.path.exists('/kaggle/input')
    
    if IS_KAGGLE:
        INPUT_PATH = '/kaggle/input/arc-prize-2025/arc-agi_test_challenges.json'
        OUTPUT_PATH = '/kaggle/working/submission.json'
        LOG_PATH = '/kaggle/working/log.txt'
    else:
        INPUT_PATH = 'data/arc-agi_test_challenges.json'
        OUTPUT_PATH = 'submission.json'
        LOG_PATH = 'log.txt'

config = ARC2025Config()

# ═══════════════════════════════════════════════════════════════════════════════
# PERFORMANCE TRACKER (Built-in Infrastructure)
# ═══════════════════════════════════════════════════════════════════════════════

@dataclass
class SolverStats:
    name: str
    triggers: int = 0
    attempts: int = 0
    task_ids: List[str] = field(default_factory=list)

class PerformanceTracker:
    def __init__(self):
        self.solvers: Dict[str, SolverStats] = {}
        self.total_tasks = 0
        self.start_time = time.time()

    def record_trigger(self, solver_name: str, task_id: str):
        if solver_name not in self.solvers:
            self.solvers[solver_name] = SolverStats(name=solver_name)
        stats = self.solvers[solver_name]
        stats.triggers += 1
        if task_id not in stats.task_ids:
            stats.task_ids.append(task_id)

    def set_total_tasks(self, total: int):
        self.total_tasks = total

    def get_coverage(self, solver_name: str) -> float:
        if solver_name not in self.solvers or self.total_tasks == 0:
            return 0.0
        return self.solvers[solver_name].triggers / self.total_tasks

    def get_stats_summary(self) -> Dict:
        summary = {}
        for name, stats in self.solvers.items():
            summary[name] = {
                'triggers': stats.triggers,
                'coverage': self.get_coverage(name)
            }
        return summary

# Global performance tracker
perf_tracker = PerformanceTracker()

# ═══════════════════════════════════════════════════════════════════════════════
# LOGGING SYSTEM
# ═══════════════════════════════════════════════════════════════════════════════

class Logger:
    def __init__(self, log_path: str):
        self.log_path = log_path
        self.start_time = time.time()
        with open(self.log_path, 'w') as f:
            f.write(f"UberOrca SubtleGenius v5 - Infrastructure Edition\n")
            f.write(f"Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"Environment: {'Kaggle' if config.IS_KAGGLE else 'Local'}\n")
            f.write("="*70 + "\n\n")
    
    def log(self, message: str, console: bool = True):
        timestamp = time.time() - self.start_time
        log_line = f"[{timestamp:8.2f}s] {message}\n"
        with open(self.log_path, 'a') as f:
            f.write(log_line)
        if console:
            print(message)

logger = Logger(config.LOG_PATH)
logger.log(f"✅ Configuration loaded")
logger.log(f"✅ Performance tracker initialized")
logger.log(f"   Environment: {'Kaggle' if config.IS_KAGGLE else 'Local'}")
logger.log(f"   Output: {config.OUTPUT_PATH}\n")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 2: SUBMISSION VALIDATOR
# ═══════════════════════════════════════════════════════════════════════════════

class SubmissionValidator:
    @staticmethod
    def validate_grid(grid: List[List[int]], context: str = "") -> Tuple[bool, str]:
        if not isinstance(grid, list):
            return False, f"{context}: Grid must be list"
        if len(grid) == 0:
            return False, f"{context}: Grid cannot be empty"
        if not all(isinstance(row, list) for row in grid):
            return False, f"{context}: Grid must be 2D list"
        if len(grid) > 0:
            row_len = len(grid[0])
            if not all(len(row) == row_len for row in grid):
                return False, f"{context}: Ragged array"
        for i, row in enumerate(grid):
            for j, val in enumerate(row):
                if not isinstance(val, (int, np.integer)):
                    return False, f"{context}[{i}][{j}]: Must be int"
                if val < 0 or val > 9:
                    return False, f"{context}[{i}][{j}]: Value {val} not in 0-9"
        return True, "Valid"
    
    @staticmethod
    def validate_submission(submission: Dict, test_challenges: Dict) -> Tuple[bool, str]:
        logger.log("\n" + "="*70)
        logger.log("🔍 VALIDATING SUBMISSION")
        logger.log("="*70)
        
        if not isinstance(submission, dict):
            return False, f"❌ Submission must be DICT"
        
        logger.log(f"✅ Structure: Dictionary")
        
        test_task_ids = set(test_challenges.keys())
        submission_task_ids = set(submission.keys())
        
        missing_tasks = test_task_ids - submission_task_ids
        if missing_tasks:
            return False, f"❌ Missing tasks: {list(missing_tasks)[:5]}"
        
        logger.log(f"✅ Task coverage: {len(submission_task_ids)} tasks")
        
        errors = []
        for task_id, predictions in submission.items():
            if not isinstance(predictions, list):
                errors.append(f"Task {task_id}: predictions must be list")
                continue
            
            for idx, pred in enumerate(predictions):
                if "attempt_1" not in pred or "attempt_2" not in pred:
                    errors.append(f"Task {task_id}[{idx}]: Missing attempts")
                    continue
                
                for attempt_key in ["attempt_1", "attempt_2"]:
                    is_valid, msg = SubmissionValidator.validate_grid(
                        pred[attempt_key], f"{task_id}[{idx}][{attempt_key}]"
                    )
                    if not is_valid:
                        errors.append(msg)
            
            if len(errors) >= 5:
                break
        
        if errors:
            return False, f"❌ Errors:\n" + "\n".join(errors[:5])
        
        logger.log(f"✅ All predictions valid")
        logger.log("="*70)
        logger.log("🎉 VALIDATION PASSED")
        logger.log("="*70 + "\n")
        
        return True, "Valid"

logger.log("✅ Validator loaded")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 3: SAFE DEFAULTS
# ═══════════════════════════════════════════════════════════════════════════════

class SafeDefaults:
    @staticmethod
    def copy_input(task_data: Dict, test_idx: int = 0) -> List[List[int]]:
        try:
            return task_data['test'][test_idx]['input']
        except:
            return [[0]]
    
    @staticmethod
    def convert_to_python_list(grid):
        if isinstance(grid, np.ndarray):
            return grid.tolist()
        return grid

logger.log("✅ Safe defaults loaded")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 4: MEGA SOLVER - ALL ITERATIONS + INFRASTRUCTURE INTEGRATED
# ═══════════════════════════════════════════════════════════════════════════════

# ═════════════════════════════════════════════════════════════════════════════════
# UTILITY FUNCTIONS
# ═════════════════════════════════════════════════════════════════════════════════

def grid_to_tuple(grid: List[List[int]]) -> Tuple:
    return tuple(tuple(row) for row in grid)

def grids_equal(g1: List[List[int]], g2: List[List[int]]) -> bool:
    return np.array_equal(np.array(g1), np.array(g2))

def shape(grid: List[List[int]]) -> Tuple[int, int]:
    if not grid:
        return (0, 0)
    return (len(grid), len(grid[0]) if grid[0] else 0)

def flatten(grid: List[List[int]]) -> List[int]:
    return [val for row in grid for val in row]

# ═════════════════════════════════════════════════════════════════════════════════
# OBJECT DETECTION - FIXED VERSION (Detection + Transformation Together)
# ═════════════════════════════════════════════════════════════════════════════════

@dataclass
class DetectedObject:
    id: int
    color: int
    pixels: List[Tuple[int, int]]
    bounding_box: Tuple[int, int, int, int]
    @property
    def area(self) -> int:
        return len(self.pixels)

def find_connected_components(grid: List[List[int]], background_color: int = 0) -> List[DetectedObject]:
    arr = np.array(grid)
    h, w = arr.shape
    visited = np.zeros((h, w), dtype=bool)
    objects = []
    obj_id = 0
    
    for i in range(h):
        for j in range(w):
            if visited[i, j] or arr[i, j] == background_color:
                continue
            color = arr[i, j]
            pixels = []
            stack = [(i, j)]
            
            while stack:
                r, c = stack.pop()
                if r < 0 or r >= h or c < 0 or c >= w:
                    continue
                if visited[r, c] or arr[r, c] != color:
                    continue
                visited[r, c] = True
                pixels.append((r, c))
                stack.extend([(r-1, c), (r+1, c), (r, c-1), (r, c+1)])
            
            if pixels:
                rows = [p[0] for p in pixels]
                cols = [p[1] for p in pixels]
                bbox = (min(rows), min(cols), max(rows), max(cols))
                objects.append(DetectedObject(obj_id, color, pixels, bbox))
                obj_id += 1
    
    return objects

def detect_object_color_change(task_data: Dict) -> Optional[Dict]:
    train_pairs = task_data.get('train', [])
    if len(train_pairs) < 1:
        return None
    try:
        color_mappings = []
        for pair in train_pairs:
            inp = pair['input']
            out = pair['output']
            if np.array(inp).shape != np.array(out).shape:
                return None
            inp_objects = find_connected_components(inp)
            out_objects = find_connected_components(out)
            if len(inp_objects) != len(out_objects):
                return None
            pair_mapping = {}
            for inp_obj, out_obj in zip(inp_objects, out_objects):
                if sorted(inp_obj.pixels) != sorted(out_obj.pixels):
                    return None
                if inp_obj.color in pair_mapping:
                    if pair_mapping[inp_obj.color] != out_obj.color:
                        return None
                else:
                    pair_mapping[inp_obj.color] = out_obj.color
            color_mappings.append(pair_mapping)
        if not color_mappings:
            return None
        first_mapping = color_mappings[0]
        if not all(m == first_mapping for m in color_mappings):
            return None
        if all(k == v for k, v in first_mapping.items()):
            return None
        return {'type': 'object_color_change', 'color_mapping': first_mapping, 'confidence': 0.85}
    except:
        return None

def apply_object_color_change(test_input: List[List[int]], color_mapping: Dict[int, int]) -> List[List[int]]:
    arr = np.array(test_input)
    result = arr.copy()
    for old_color, new_color in color_mapping.items():
        result[arr == old_color] = new_color
    return result.tolist()

# ═════════════════════════════════════════════════════════════════════════════════
# RULE INDUCTION (Iteration 4) - HIGHEST PRIORITY
# ═════════════════════════════════════════════════════════════════════════════════

def detect_color_mapping_rule(train_pairs: List[Dict]) -> Optional[Dict]:
    if not train_pairs:
        return None
    try:
        inp = train_pairs[0]['input']
        out = train_pairs[0]['output']
        if shape(inp) != shape(out):
            return None
        mapping = {}
        for (in_val, out_val) in zip(flatten(inp), flatten(out)):
            if in_val in mapping:
                if mapping[in_val] != out_val:
                    return None
            else:
                mapping[in_val] = out_val
        if all(k == v for k, v in mapping.items()):
            return None
        for pair in train_pairs[1:]:
            if shape(pair['input']) != shape(pair['output']):
                return None
            for (in_val, out_val) in zip(flatten(pair['input']), flatten(pair['output'])):
                if mapping.get(in_val, in_val) != out_val:
                    return None
        return {'type': 'color_mapping', 'mapping': mapping, 'confidence': 0.90}
    except:
        return None

def apply_color_mapping_rule(grid: List[List[int]], mapping: Dict[int, int]) -> List[List[int]]:
    result = []
    for row in grid:
        new_row = [mapping.get(val, val) for val in row]
        result.append(new_row)
    return result

def detect_size_tile_rule(train_pairs: List[Dict]) -> Optional[Dict]:
    if not train_pairs:
        return None
    try:
        ratios = []
        for pair in train_pairs:
            inp_h, inp_w = shape(pair['input'])
            out_h, out_w = shape(pair['output'])
            if inp_h == 0 or inp_w == 0:
                return None
            ratios.append((out_h / inp_h, out_w / inp_w))
        if not all(r == ratios[0] for r in ratios):
            return None
        h_ratio, w_ratio = ratios[0]
        if h_ratio != int(h_ratio) or w_ratio != int(w_ratio) or (h_ratio == 1.0 and w_ratio == 1.0):
            return None
        h_scale, w_scale = int(h_ratio), int(w_ratio)
        for pair in train_pairs:
            expected = np.tile(np.array(pair['input']), (h_scale, w_scale))
            if not np.array_equal(expected, np.array(pair['output'])):
                return None
        return {'type': 'size_tile', 'h_scale': h_scale, 'w_scale': w_scale, 'confidence': 0.95}
    except:
        return None

def apply_size_tile_rule(grid: List[List[int]], h_scale: int, w_scale: int) -> List[List[int]]:
    return np.tile(np.array(grid), (h_scale, w_scale)).tolist()

def detect_rule_induction(task_data: Dict) -> Optional[Tuple[str, Callable, float]]:
    train_pairs = task_data.get('train', [])
    if not train_pairs:
        return None
    rule = detect_color_mapping_rule(train_pairs)
    if rule:
        mapping = rule['mapping']
        return ("rule_color_map", lambda g: apply_color_mapping_rule(g, mapping), rule['confidence'])
    rule = detect_size_tile_rule(train_pairs)
    if rule:
        h_scale, w_scale = rule['h_scale'], rule['w_scale']
        return ("rule_tile", lambda g: apply_size_tile_rule(g, h_scale, w_scale), rule['confidence'])
    return None

# ═════════════════════════════════════════════════════════════════════════════════
# PATTERN MATCHING (Iteration 1)
# ═════════════════════════════════════════════════════════════════════════════════

def rotate_90_cw(grid: List[List[int]]) -> List[List[int]]:
    return np.rot90(np.array(grid), k=-1).tolist()

def flip_horizontal(grid: List[List[int]]) -> List[List[int]]:
    return np.fliplr(np.array(grid)).tolist()

def flip_vertical(grid: List[List[int]]) -> List[List[int]]:
    return np.flipud(np.array(grid)).tolist()

def detect_color_mapping(input_grid: List[List[int]], output_grid: List[List[int]]) -> Optional[Dict[int, int]]:
    inp = np.array(input_grid)
    out = np.array(output_grid)
    if inp.shape != out.shape:
        return None
    mapping = {}
    for in_val, out_val in zip(inp.flatten(), out.flatten()):
        if in_val in mapping and mapping[in_val] != out_val:
            return None
        mapping[in_val] = out_val
    return mapping if not all(k == v for k, v in mapping.items()) else None

def apply_color_mapping(grid: List[List[int]], mapping: Dict[int, int]) -> List[List[int]]:
    arr = np.array(grid)
    result = arr.copy()
    for old_color, new_color in mapping.items():
        result[arr == old_color] = new_color
    return result.tolist()

def detect_pattern_matching(task_data: Dict) -> Optional[Tuple[str, Callable]]:
    train_pairs = task_data.get('train', [])
    if len(train_pairs) < 1:
        return None
    transformations = [
        ("rotate_90_cw", rotate_90_cw),
        ("rotate_180", lambda g: rotate_90_cw(rotate_90_cw(g))),
        ("flip_horizontal", flip_horizontal),
        ("flip_vertical", flip_vertical),
    ]
    for name, transform in transformations:
        if all(grids_equal(transform(pair['input']), pair['output']) for pair in train_pairs):
            return (name, transform)
    first_mapping = detect_color_mapping(train_pairs[0]['input'], train_pairs[0]['output'])
    if first_mapping:
        if all(detect_color_mapping(pair['input'], pair['output']) == first_mapping for pair in train_pairs):
            return ("color_mapping", lambda g: apply_color_mapping(g, first_mapping))
    return None

# ═════════════════════════════════════════════════════════════════════════════════
# SYMMETRY (Iteration 3)
# ═════════════════════════════════════════════════════════════════════════════════

def detect_symmetry(grid: List[List[int]]) -> Optional[str]:
    arr = np.array(grid)
    h, w = arr.shape
    left = arr[:, :w//2]
    right = np.fliplr(arr[:, w//2:])
    if left.shape == right.shape and 0.6 <= np.mean(left == right) < 0.95:
        return "horizontal"
    top = arr[:h//2, :]
    bottom = np.flipud(arr[h//2:, :])
    if top.shape == bottom.shape and 0.6 <= np.mean(top == bottom) < 0.95:
        return "vertical"
    return None

def complete_symmetry(grid: List[List[int]], sym_type: str) -> List[List[int]]:
    arr = np.array(grid)
    h, w = arr.shape
    if sym_type == "horizontal":
        left = arr[:, :w//2]
        return np.hstack([left, np.fliplr(left)]).tolist()
    elif sym_type == "vertical":
        top = arr[:h//2, :]
        return np.vstack([top, np.flipud(top)]).tolist()
    return grid

# ═════════════════════════════════════════════════════════════════════════════════
# ENSEMBLE SYSTEM WITH PERFORMANCE TRACKING
# ═════════════════════════════════════════════════════════════════════════════════

@dataclass
class SolverPrediction:
    grid: List[List[int]]
    confidence: float
    solver_name: str

def collect_predictions(test_input: List[List[int]], task_data: Dict, task_id: str) -> List[SolverPrediction]:
    predictions = []
    
    # PRIORITY 1: Rule Induction
    try:
        result = detect_rule_induction(task_data)
        if result:
            name, transform, conf = result
            pred = SafeDefaults.convert_to_python_list(transform(test_input))
            predictions.append(SolverPrediction(pred, conf, name))
            perf_tracker.record_trigger(name, task_id)
    except:
        pass
    
    # PRIORITY 2: Pattern matching
    try:
        result = detect_pattern_matching(task_data)
        if result:
            name, transform = result
            pred = SafeDefaults.convert_to_python_list(transform(test_input))
            predictions.append(SolverPrediction(pred, 0.85, f"pattern_{name}"))
            perf_tracker.record_trigger(f"pattern_{name}", task_id)
    except:
        pass
    
    # PRIORITY 3: Object transformations (FIXED)
    try:
        rule = detect_object_color_change(task_data)
        if rule and rule['type'] == 'object_color_change':
            pred = apply_object_color_change(test_input, rule['color_mapping'])
            predictions.append(SolverPrediction(pred, rule['confidence'], 'object_color_change'))
            perf_tracker.record_trigger('object_color_change', task_id)
    except:
        pass
    
    # PRIORITY 4: Symmetry completion
    try:
        sym_type = detect_symmetry(test_input)
        if sym_type:
            pred = SafeDefaults.convert_to_python_list(complete_symmetry(test_input, sym_type))
            predictions.append(SolverPrediction(pred, 0.75, f"symmetry_{sym_type}"))
            perf_tracker.record_trigger(f"symmetry_{sym_type}", task_id)
    except:
        pass
    
    return predictions

def vote_on_predictions(predictions: List[SolverPrediction], attempt: int, test_input: List[List[int]]) -> List[List[int]]:
    if not predictions:
        return test_input
    vote_groups = {}
    for pred in predictions:
        grid_key = grid_to_tuple(pred.grid)
        if grid_key not in vote_groups:
            vote_groups[grid_key] = {"grid": pred.grid, "total_confidence": 0.0, "solvers": []}
        vote_groups[grid_key]["total_confidence"] += pred.confidence
        vote_groups[grid_key]["solvers"].append(pred.solver_name)
    ranked = sorted(vote_groups.values(), key=lambda x: x["total_confidence"], reverse=True)
    if attempt == 1:
        return ranked[0]["grid"]
    else:
        if ranked[0]["total_confidence"] > 0.95:
            return ranked[0]["grid"]
        return ranked[1]["grid"] if len(ranked) > 1 else test_input

def ensemble_solver(test_input: List[List[int]], task_data: Dict, attempt: int = 1, task_id: str = "") -> List[List[int]]:
    try:
        predictions = collect_predictions(test_input, task_data, task_id)
        if predictions:
            solver_names = [p.solver_name for p in predictions]
            logger.log(f"  Task {task_id}: {len(predictions)} solvers → {solver_names}", console=False)
        result = vote_on_predictions(predictions, attempt, test_input)
        return SafeDefaults.convert_to_python_list(result)
    except Exception as e:
        logger.log(f"  ⚠️  Task {task_id} error: {str(e)[:60]}", console=False)
        return test_input

logger.log("✅ Mega Solver loaded (v5 - Infrastructure Edition)")
logger.log("   - Rule Induction (HIGHEST PRIORITY)")
logger.log("   - Pattern Matching")
logger.log("   - Object Transformations (FIXED)")
logger.log("   - Symmetry Completion")
logger.log("   - Performance Tracking (ENABLED)\n")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 5: SUBMISSION GENERATOR WITH PERFORMANCE TRACKING
# ═══════════════════════════════════════════════════════════════════════════════

def generate_submission(test_challenges: Dict) -> Dict:
    logger.log("\n" + "="*70)
    logger.log("🔨 GENERATING SUBMISSION")
    logger.log("="*70)
    
    submission = {}
    total_tasks = len(test_challenges)
    perf_tracker.set_total_tasks(total_tasks)
    start_time = time.time()
    
    for task_idx, (task_id, task_data) in enumerate(test_challenges.items(), 1):
        if task_idx % 50 == 0:
            elapsed = time.time() - start_time
            rate = task_idx / elapsed if elapsed > 0 else 0
            remaining = (total_tasks - task_idx) / rate if rate > 0 else 0
            logger.log(f"  Progress: {task_idx}/{total_tasks} ({task_idx/total_tasks*100:.1f}%) | "
                      f"Rate: {rate:.1f} tasks/sec | ETA: {remaining/60:.1f}min")
        
        task_predictions = []
        for test_idx in range(len(task_data['test'])):
            test_input = task_data['test'][test_idx]['input']
            attempt_1 = ensemble_solver(test_input, task_data, attempt=1, task_id=task_id)
            attempt_2 = ensemble_solver(test_input, task_data, attempt=2, task_id=task_id)
            task_predictions.append({
                "attempt_1": SafeDefaults.convert_to_python_list(attempt_1),
                "attempt_2": SafeDefaults.convert_to_python_list(attempt_2)
            })
        submission[task_id] = task_predictions
    
    elapsed = time.time() - start_time
    logger.log(f"\n✅ Generated {total_tasks} tasks in {elapsed:.1f}s")
    logger.log("="*70 + "\n")
    
    # Performance report
    logger.log("\n" + "="*70)
    logger.log("📊 PERFORMANCE STATISTICS")
    logger.log("="*70)
    
    stats = perf_tracker.get_stats_summary()
    if stats:
        logger.log(f"  {'Solver':<30} {'Triggers':>10} {'Coverage':>12}")
        logger.log(f"  {'-'*30} {'-'*10} {'-'*12}")
        for solver, data in sorted(stats.items(), key=lambda x: x[1]['coverage'], reverse=True):
            logger.log(f"  {solver:<30} {data['triggers']:>10} {data['coverage']*100:>11.1f}%")
    
    logger.log("="*70 + "\n")
    
    return submission

logger.log("✅ Submission generator loaded")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 6: MAIN EXECUTION
# ═══════════════════════════════════════════════════════════════════════════════

def main():
    logger.log("\n" + "="*70)
    logger.log("🐋 UBERORCA SUBTLEGENIUS V5 - THE STIG EDITION")
    logger.log("="*70)
    logger.log(f"Environment: {'Kaggle' if config.IS_KAGGLE else 'Local'}")
    logger.log(f"Output: {config.OUTPUT_PATH}")
    logger.log("="*70 + "\n")
    
    logger.log("📂 Loading test challenges...")
    try:
        with open(config.INPUT_PATH, 'r') as f:
            test_challenges = json.load(f)
        logger.log(f"✅ Loaded {len(test_challenges)} tasks\n")
    except Exception as e:
        logger.log(f"❌ Failed to load: {e}")
        return
    
    submission = generate_submission(test_challenges)
    
    is_valid, msg = SubmissionValidator.validate_submission(submission, test_challenges)
    if not is_valid:
        logger.log(f"❌ VALIDATION FAILED: {msg}")
        return
    
    logger.log("💾 Saving submission...")
    with open(config.OUTPUT_PATH, 'w') as f:
        json.dump(submission, f, indent=2)
    
    file_size = os.path.getsize(config.OUTPUT_PATH)
    logger.log(f"✅ Saved: {config.OUTPUT_PATH} ({file_size:,} bytes)")
    
    logger.log("\n" + "="*70)
    logger.log("🎉 EXECUTION COMPLETE - THE STIG HAS DRIVEN")
    logger.log("="*70)
    logger.log(f"Submission: {config.OUTPUT_PATH}")
    logger.log(f"Log: {config.LOG_PATH}")
    logger.log(f"Status: ✅ READY FOR KAGGLE")
    logger.log("="*70 + "\n")
    
    print(f"\n📝 Log: {config.LOG_PATH}")
    print(f"📊 Submission: {config.OUTPUT_PATH}")
    print(f"✅ UNLEASHED!\n")

# RUN IT
main()